In [ ]:
!pip install paho_mqtt

In [ ]:
!pip install pymysql

In [35]:
import paho.mqtt.client as mqtt
client = mqtt.Client()
client.username_pw_set(username = "mirae", password = "daejeon1234")
client.connect('34.64.86.80', 19883)  # MQTT 브로커에 연결, IP 주소와 포트번호 지정
client.publish('house/light', '빨리켜지기')

In [ ]:
import pymysql
import json
con = pymysql.connect(host='34.64.86.80', port = 3306, user = 'root', passwd = 'acorn1234',
                     db = 'mqtt', charset = 'utf8', autocommit=True)
cursor = con.cursor()

def on_connect(client, userdata, flags, rc):
    print("연결되었습니다 " + str(rc))

def on_message(client, userdata, msg):
    payload = msg.payload.decode('utf-8')
    parsed_data = json.loads(payload)
    
    light = parsed_data.get("light")
    door = parsed_data.get("door")

    cursor.execute('insert into mqtt(light, door) values (%s, %s)', (light, door))
    con.commit()
    if(cursor.rowcount):
      print("데이터 삽입 성공!")
    else:
      print("데이터 삽입 실패!")

def on_close(client, userdata, rc):
  print("종료")
  con.close()

client = mqtt.Client()
client.username_pw_set(username = 'mirae', password='daejeon1234')
client.on_connect = on_connect
client.on_message = on_message
client.on_close = on_close
client.connect('34.64.86.80', 19883)
client.subscribe('house/light', qos=1)
client.loop_forever()

In [ ]:
# 문제
# 수신한 데이터를 mariadb 테이블에 저장
# publish되면, subscribe하여 mariadb에 저장되게 하기

# == 서버부 코드 == #
import json   # web 표준 (네트워크로 데이터를 전송)

message = {}
message["light"] = "on"
message["door"] = "close"
transferdata = json.dumps(message)
client.connect('34.64.86.80', 19883)
client.publish("house/light", transferdata)